## CollapseView Op

In [ ]:
//load ImageJ
%classpath config resolver imagej.public https://maven.imagej.net/content/groups/public
%classpath add jar /home/gabe/code/imagej/imagej-common/target/imagej-common-0.27.0-SNAPSHOT.jar
%classpath add jar /home/gabe/code/gselzer/imagej-notebook/target/imagej-notebook-0.3.0-SNAPSHOT.jar
%classpath add mvn net.imagej imagej 2.0.0-rc-67

//create ImageJ object
ij = new net.imagej.ImageJ()

This `Op` wraps the `Views.collapse()` method of ImgLib2, collapsing the n-th dimension of a `RandomAccessible` or `RandomAccessibleInterval` into a [`Composite`](http://javadoc.scijava.org/ImgLib2/net/imglib2/view/composite/Composite.html), returning a (n-1) dimensional `RandomAccessible` or `RandomAccessibleInterval` or type `Composite`.

In [ ]:
ij.op().help('collapseView')

One situation in which `collapseView` can be helpful is in the case that you have an image with 3 channels that you would like to convert to `ARGBType`:

In [ ]:
input = ij.scifio().datasetIO().open("http://imagej.net/images/clown.png")

input

Note that this image has 3 channels:

In [ ]:
import net.imglib2.util.Intervals

Arrays.toString( Intervals.dimensionsAsIntArray( input ) ) 

Let's convert this image to `ARGBType`. First we have to collapse the image into `Composite`s using [`collapseView`](collapseView.ipynb). Note that we have to cast our `Img` to a `RandomAccessibleInterval`. This is because `Img` extends both `RandomAccessibleInterval` and `RandomAccessible`, thus causing confusion on the part of the `Op` matcher:

In [ ]:
import net.imglib2.view.Views

collapsed = ij.op().run("collapseView", input);

//note that collapsed contains GenericComposites at each pixel, so we cannot display it (thus needing the null)
null

Now that we have our collapsed image we can use ImgLib2's [`Converter`](http://javadoc.scijava.org/ImgLib2/net/imglib2/converter/Converter.html) to take the values of the `GenericComposite` image from `collapseView` and convert them into `ARGBType`:

In [ ]:
import net.imglib2.converter.Converter
import net.imglib2.converter.Converters
import net.imglib2.RandomAccessibleInterval
import net.imglib2.type.numeric.ARGBType
import net.imglib2.type.numeric.integer.UnsignedByteType
import net.imglib2.view.composite.GenericComposite

//create the GenericComposite to ARGBType Converter
converter = new Converter<GenericComposite<UnsignedByteType>, ARGBType>() {
    @Override
    public void convert(GenericComposite<UnsignedByteType> input, ARGBType output) {
        red = input.get(0).get()
        green = input.get(1).get()
        blue = input.get(2).get()
        alpha = 0
        value = output.rgba(red, green, blue, alpha)
        output.set(value)
    }
}

//use Converters and our new converter above to convert the GenericComposite image into an ARGBType image
converted = Converters.convert( collapsed, converter, new ARGBType() )

Now that we have our `ARGBType` image, let's check the dimensions again:

In [ ]:
import net.imglib2.util.Intervals

Arrays.toString( Intervals.dimensionsAsIntArray( converted ) ) 

Note that we have one dimension fewer than when we started before.